## Model generator main file

In [ ]:
import numpy as np
import re
import unidecode
import os
import pickle
import sys

import matplotlib.pyplot as plt
%matplotlib inline

from keras                  import regularizers
from keras                  import Input, Model, Sequential
from keras.layers           import Bidirectional, Activation, TimeDistributed, Dense, RepeatVector, Embedding, Dropout, BatchNormalization
from keras.layers.recurrent import LSTM, GRU, SimpleRNN
from keras.utils            import np_utils
from keras.callbacks        import EarlyStopping, TensorBoard, ModelCheckpoint

Import data

In [ ]:
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'

with open(data_path + '/joint_angle_data.pickle', 'rb') as file:
    joint_angle_data = pickle.load(file)
    file.close()
    
with open(data_path + '/power_data.pickle', 'rb') as file:
    power_data = pickle.load(file)
    file.close()